In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
train = pd.read_csv('sales_train_v2.csv', parse_dates=['date'],
                    date_parser=lambda x: pd.to_datetime(x, format='%d.%m.%Y'))
test = pd.read_csv('test.csv')
items = pd.read_csv('items.csv')
item_categories = pd.read_csv('item_categories.csv')
shops = pd.read_csv('shops.csv')

In [18]:
def eda(data):
    print("---------------------------------------------------Data Head----------------------------------------------------")
    print(data.head(5))
    print("---------------------------------------------------Information--------------------------------------------------")
    print(data.info())
    print("---------------------------------------------------Data Types---------------------------------------------------")
    print(data.dtypes)
    print("---------------------------------------------------Missing value------------------------------------------------")
    print(data.isnull().sum())
    print("---------------------------------------------------Null value---------------------------------------------------")
    print(data.isna().sum())
    print("---------------------------------------------------Shape of Data------------------------------------------------")
    print(data.shape)

In [19]:
eda(train)

---------------------------------------------------Data Head----------------------------------------------------
        date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0 2013-01-02               0       59    22154      999.00           1.0
1 2013-01-03               0       25     2552      899.00           1.0
2 2013-01-05               0       25     2552      899.00          -1.0
3 2013-01-06               0       25     2554     1709.05           1.0
4 2013-01-15               0       25     2555     1099.00           1.0
---------------------------------------------------Information--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
date              datetime64[ns]
date_block_num    int64
shop_id           int64
item_id           int64
item_price        float64
item_cnt_day      float64
dtypes: datetime64[ns](1), float64(2), int64(3)
memory usage: 134.4

In [20]:
eda(test)

---------------------------------------------------Data Head----------------------------------------------------
   ID  shop_id  item_id
0   0        5     5037
1   1        5     5320
2   2        5     5233
3   3        5     5232
4   4        5     5268
---------------------------------------------------Information--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 3 columns):
ID         214200 non-null int64
shop_id    214200 non-null int64
item_id    214200 non-null int64
dtypes: int64(3)
memory usage: 4.9 MB
None
---------------------------------------------------Data Types---------------------------------------------------
ID         int64
shop_id    int64
item_id    int64
dtype: object
---------------------------------------------------Missing value------------------------------------------------
ID         0
shop_id    0
item_id    0
dtype: int64
-----------------------------------

In [21]:
eda(items)

---------------------------------------------------Data Head----------------------------------------------------
                                           item_name  item_id  \
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0   
1  !ABBYY FineReader 12 Professional Edition Full...        1   
2      ***В ЛУЧАХ СЛАВЫ   (UNV)                    D        2   
3    ***ГОЛУБАЯ ВОЛНА  (Univ)                      D        3   
4        ***КОРОБКА (СТЕКЛО)                       D        4   

   item_category_id  
0                40  
1                76  
2                40  
3                40  
4                40  
---------------------------------------------------Information--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22170 entries, 0 to 22169
Data columns (total 3 columns):
item_name           22170 non-null object
item_id             22170 non-null int64
item_category_id    22170 non-null int64
dtypes: int64(2), o

In [22]:
eda(item_categories)

---------------------------------------------------Data Head----------------------------------------------------
        item_category_name  item_category_id
0  PC - Гарнитуры/Наушники                 0
1         Аксессуары - PS2                 1
2         Аксессуары - PS3                 2
3         Аксессуары - PS4                 3
4         Аксессуары - PSP                 4
---------------------------------------------------Information--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 2 columns):
item_category_name    84 non-null object
item_category_id      84 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.4+ KB
None
---------------------------------------------------Data Types---------------------------------------------------
item_category_name    object
item_category_id       int64
dtype: object
---------------------------------------------------Missing value-----------------

In [23]:
eda(shops)

---------------------------------------------------Data Head----------------------------------------------------
                        shop_name  shop_id
0   !Якутск Орджоникидзе, 56 фран        0
1   !Якутск ТЦ "Центральный" фран        1
2                Адыгея ТЦ "Мега"        2
3  Балашиха ТРК "Октябрь-Киномир"        3
4        Волжский ТЦ "Волга Молл"        4
---------------------------------------------------Information--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
shop_name    60 non-null object
shop_id      60 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.0+ KB
None
---------------------------------------------------Data Types---------------------------------------------------
shop_name    object
shop_id       int64
dtype: object
---------------------------------------------------Missing value------------------------------------------------
shop_name    0
s